# Distributed Training Notebook

In [ ]:
# from absl import app
# from absl import flags
# from absl import logging
from google.cloud import aiplatform as vertex_ai
import os
import time

import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_io as tfio

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

In [ ]:
PREFIX = 'spotify-2tower'
TB_DISPLAY_NAME = f'{PREFIX}-tensorboard'
PROJECT= 'hybrid-vertex'
REGION='us-central1'


## Initialize the Vertex SDK


In [ ]:
# initialize vertex sdk
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### Perepare Vertex Training Package



In [ ]:
# Make folder for Python training script
! rm -rf vertex_train/trainer
! mkdir vertex_train/trainer

In [ ]:
%%writefile vertex_train/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf-gpu.2-9

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip install -r trainer/requirements.txt

# Sets up the entry point to invoke the trainer.
# ENTRYPOINT ["python", "-m", "trainer.task"]

Requirements

In [ ]:
%%writefile vertex_train/trainer/requirements.txt

tensorflow-recommenders==0.7.0
tensorflow==2.9.2
tensorboard==2.9.1
tensorboard-data-server==0.6.1
tensorboard-plugin-profile==2.5.0
cloudml-hypertune